In [1]:
import cv2
import numpy as np
import os

from keras.models import Sequential, Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input, Add, Dense, Activation, ZeroPadding3D, BatchNormalization, Flatten, Conv3D, MaxPooling3D, GlobalAveragePooling3D
from keras.preprocessing.image import ImageDataGenerator

from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.utils import Sequence

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.test import gpu_device_name

from os.path import join
from random import randint

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def convolutional_block(X, F, N, stage):   
    
    name_conv = "conv" + str(stage)
    name_bn = "bn" + str(stage)
    F1, F2, F3 = F
    
    for n in range(N):
        n = str(n)
        X_shortcut = X
        
        s = (2, 2, 2) if n == "0" and stage != 2 else (1, 1, 1)
       

        X = Conv3D(filters=F1, kernel_size=(1, 1, 1), strides=s, padding='same', name=name_conv + "_a_" + n, kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=name_bn + "_a_" + n)(X)
        X = Activation('relu')(X)

        X = Conv3D(filters=F2, kernel_size=(3, 3, 3), strides=(1, 1, 1), padding='same', name=name_conv + "_b_" + n, kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(name=name_bn + "_b_" + n)(X)
        X = Activation('relu')(X)
        
        X = Conv3D(filters=F3, kernel_size=(1, 1, 1), strides=(1, 1, 1), padding='same', name=name_conv + "_c_" + n, kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(name=name_bn + "_c_" + n)(X)
        X = Activation('relu')(X)

        X_shortcut = Conv3D(filters=F3, kernel_size=(1, 1, 1), strides=s, padding='same', name=name_conv + "_shortcut_" + n, kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
        X_shortcut = BatchNormalization(name=name_bn + "_shortcut_" + n)(X_shortcut)

        X = Add()([X, X_shortcut])
        X = Activation('relu')(X)
        
   # print("X_conv{} : ".format(stage), X.shape)

    return X

    

def ResNet50_3D(input_shape=(16, 112, 112, 3), output_nb_classes=10):

    X_input = Input(input_shape)
    print("input : ", X_input.shape)
    X = ZeroPadding3D((1, 3, 3))(X_input)
    
    print("zeroPadding : ",X.shape)
    #conv1
    X = Conv3D(64, (3, 7, 7), strides=(1, 2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(name='bn_conv1')(X)
    print("X_conv1 : ", X.shape)
    
    X = MaxPooling3D((3, 3, 3), strides=(2, 2, 2), padding="same")(X)
    print("X_pool : ", X.shape)
    
    #conv2
    X = convolutional_block(X, F=[128, 128, 256], N=3, stage=2)
    print("X_conv2 : ", X.shape)
    
    #conv3
    X = convolutional_block(X, F=[256, 256, 512], N=4, stage=3)
    print("X_conv3 : ", X.shape)

    #conv4
    X = convolutional_block(X, F=[512, 512, 1024], N=6, stage=4)
    print("X_conv4 : ", X.shape)

    #conv5
    X = convolutional_block(X, F=[1024, 1024, 2048],  N=3, stage=5)
    print("X_conv5 : ", X.shape)

    #avgPool
    X = GlobalAveragePooling3D()(X)
    X = Flatten()(X)
    X = Dense(output_nb_classes, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

#compile model for jester
model = ResNet50_3D(input_shape=(16, 112, 112, 3), output_nb_classes=27)
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ["accuracy"])
model.summary()

input :  (None, 16, 112, 112, 3)
zeroPadding :  (None, 18, 118, 118, 3)
X_conv1 :  (None, 16, 56, 56, 64)
X_pool :  (None, 8, 28, 28, 64)
X_conv2 :  (None, 8, 28, 28, 256)
X_conv3 :  (None, 4, 14, 14, 512)
X_conv4 :  (None, 2, 7, 7, 1024)
X_conv5 :  (None, 1, 4, 4, 2048)
Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 16, 112, 112 0                                            
__________________________________________________________________________________________________
zero_padding3d (ZeroPadding3D)  (None, 18, 118, 118, 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv3D)                  (None, 16, 56, 56, 6 28288       zero_padding3d[0][0]             
_________________

In [6]:
class DataGeneratorClassifier(Sequence):
    
    def load_sample(self, video, id_begin_frame, id_end_frame, padding):
        sample = []
          
        for frame_id in range(id_begin_frame, id_end_frame + 1):
            img_path = os.path.join(self.dataset_path, 'videos', str(video), "{}.jpg".format(str(frame_id)))
            img = cv2.imread(img_path)
            sample.append(img)
            
        #print(padding)
        for frame_id in range(padding):
            sample.append(sample[-1])
        
        return np.array(sample)
    
    def augment_sample(self, sample):
        pack = []
        seed = randint(0, 100000)
        
        for i in range(len(sample)):
            image = self.augmenter.flow(np.array([sample[i]]), seed=seed).next()[0].astype('uint8')
            pack.append(image)
            
        return pack

    def resize_sample(self, video_data, size):
        return np.array([cv2.resize(frame, dsize=size, interpolation=cv2.INTER_CUBIC) for frame in video_data])
        
    def __init__(self, dataset_path, annotation_path, batch_size, nb_classes, augment):
        self.batch_size = batch_size
        self.dataset_path = dataset_path
        self.files = pd.read_csv(annotation_path)
        self.nb_classes = nb_classes
        self.augmenter = ImageDataGenerator(width_shift_range=[-10,10], 
                                            height_shift_range=[-10,10], 
                                            rotation_range=8, 
                                            brightness_range=[0.6,1.2], 
                                            zoom_range=[0.9,1.1])
        self.augment = augment
        

        
    def __len__(self):
        return len(self.files) // self.batch_size

    def __getitem__(self, index):
    
        begin = index * self.batch_size
        end = (index + 1) * self.batch_size
        batch_data = self.files[begin:end]
        X, y = self.__get_data(batch_data)
        return X, y
    
    def __get_data(self, batch_data):
        X = []
        y = []
        
        for i, row in batch_data.iterrows():
            
            sample = self.load_sample(row['video_id'], row['begin'], row['end'], row['padding'])
        
            if self.augment:
                sample = self.augment_sample(sample)
                
            X.append(sample)
            y.append(to_categorical(row['label_id'], num_classes=self.nb_classes))
      
        X = np.array(X)
        y = np.array(y)
        
        return X, y

In [7]:
import pandas as pd 

#dataset_path = r'C:\Users\mdavid\Desktop\LSF10'
#annotation_train = 'annotations_classifier_train.csv'
#annotation_validation = 'annotations_classifier_val.csv'

dataset_path = r'C:\Users\mdavid\Desktop\jester_dataset'
annotation_train = 'jester_annotations_classifier_train.csv'
annotation_validation = 'jester_annotations_classifier_validation.csv'

batch_size = 16

train_generator = DataGeneratorClassifier(dataset_path=dataset_path, annotation_path=join(dataset_path, annotation_train), batch_size=batch_size, nb_classes=27, augment=False)
val_generator = DataGeneratorClassifier(dataset_path=dataset_path, annotation_path=join(dataset_path, annotation_validation), batch_size=batch_size, nb_classes=27, augment=False)


In [ ]:
if gpu_device_name(): 
    print('Default GPU Device: {}'.format(gpu_device_name()))
else:
    print("Please install GPU version of TF")

model_ckpt = '../models/jester_classifier.h5'
 
#reload from checkpoint
#model = keras.models.load_model(model_ckpt)

checkpoint = ModelCheckpoint(
    filepath=model_ckpt,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

H = model.fit(train_generator, validation_data=val_generator, epochs=100, verbose=1, callbacks=[checkpoint, es])

Default GPU Device: /device:GPU:0
Epoch 1/100
 3392/15778 [=====>........................] - ETA: 2:31:59 - loss: 3.4040 - accuracy: 0.0357

In [9]:
model = keras.models.load_model('jester_classifier.h5')

score = model.evaluate(val_generator, verbose=1)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

ValueError: No model found in config file.

In [21]:
def TransferModel(output_nb_classes):
    base_model = keras.models.load_model('../models/jester_classifier.h5')

    for l in base_model.layers:
        l.trainable = False

    output = Dense(output_nb_classes, activation='softmax')(base_model.layers[-2].output)

    model = Model(base_model.input, [output])
    
    return model

model = TransferModel(output_nb_classes=10)
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ["accuracy"])

10
16
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[[7.8026247e-08 1.9604670e-07 6.1844580e-06 2.8174251e-16 3.4103523e-05
  1.8503456e-10 8.3488780e-12 5.2328578e-06 9.9995422e-01 1.0830893e-10]
 [4.6787817e-12 5.6789526e-07 9.9999881e-01 1.6755117e-10 1.7587006e-07
  7.7172713e-09 7.0791255e-11 8.5714433e-08 2.9706717e-07 7.2744992e-09]
 [7.2951458e-15 2.5035859e-12 2.2682871e-11 2.1440905e-09 5.0079553e-13
  1.5179479e-09 7.8845421e-13 8.0984976e-11 9.9935156e-01 6.4848416e-04]
 [6.1764091e-08 1.9233241e-06 8.3855817e-08 5.6583054e-09 5.0629723e-07
  9.9998617e-01 1.1173739e-06 3.7411620e-08 2.2506499e-07 9.9228682e-06]
 [9.9999893e-01 3.3923746e-07 5.0043437e-12 9.3226880e-08 1.8871579e-07
  2.6243387e-12 1.0672

In [ ]:
dataset_path = r'C:\Users\mdavid\Desktop\LSF10'
annotation_train = 'annotations_classifier_train.csv'
annotation_validation = 'annotations_classifier_val.csv'

batch_size = 16

train_generator = DataGeneratorClassifier(dataset_path=dataset_path, annotation_path=join(dataset_path, annotation_train), batch_size=batch_size, augment=False)
val_generator = DataGeneratorClassifier(dataset_path=dataset_path, annotation_path=join(dataset_path, annotation_validation), batch_size=batch_size, augment=False)

In [ ]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

model_ckpt = '../models/LSF10_classifier.h5'
 
#reload from checkpoint
#model = keras.models.load_model(model_ckpt)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
filepath=model_ckpt,
save_weights_only=False,
monitor='val_accuracy',
mode='max',
save_best_only=True,
verbose=1)
    
H = model.fit(train_generator, validation_data=val_generator, epochs=100, verbose=1, callbacks=[checkpoint])